# Phoenix DB Client Example

## You can load a dataset into a Phoenix table via the PhoenixDB library

##### First, create a database connection. Notice Phoenix credentials are saved into your CML Project as environment variables.

##### To add yours, navigate to Project Settings -> Advanced -> Add the variables for your user and restart the CML session.


In [67]:
!pip3 install -r requirements.txt

You should consider upgrading via the '/usr/local/bin/python3.6 -m pip install --upgrade pip' command.


In [68]:
import pandas as pd

In [79]:
class Db:
    def __init__(self):
        opts = {}
        opts['authentication'] = 'BASIC'
        opts['avatica_user'] = os.environ["WORKLOAD_USERNAME"]
        opts['avatica_password'] = os.environ["WORKLOAD_PASSWORD"]
        database_url = os.environ["COD_ENDPOINT"]
        self.TABLENAME = "test_table_paul"
        self.conn = phoenixdb.connect(database_url, autocommit=True,**opts)
        self.curs = self.conn.cursor()
        
    def create_linkage_table(self):
        
        query = """
        CREATE TABLE IF NOT EXISTS "COD_DEMO_TABLE" (
        age VARCHAR,
        workclass VARCHAR,
        fnlwgt VARCHAR,
        education VARCHAR,
        educationnum  VARCHAR,
        marital VARCHAR,
        occupation VARCHAR,
        relationship VARCHAR,
        race  VARCHAR,
        sex VARCHAR,
        capitalgain VARCHAR,
        capitalloss VARCHAR,
        hoursperweek  VARCHAR,
        nativecountry VARCHAR, 
        income VARCHAR,
        unique_id VARCHAR CONSTRAINT my_pk PRIMARY KEY (unique_id))
        """
        
        self.curs.execute(query)
        
    def upsert_linkage(self, data):

        sql = """upsert into "COD_DEMO_TABLE" \
             (age,workclass,fnlwgt,education,educationnum,marital,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,nativecountry,income, unique_id) \
             values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"""
        #print(data)
        self.curs.executemany(sql,data)
        self.conn.commit()
        
    def get_data(self):

        query = f"SELECT count(*) FROM COD_DEMO_TABLE"

        model.curs.execute(query)
        rows = model.curs.fetchall()

        return rows

In [80]:
def upsert_data(data, records=100):
        total_records=0
        header = True
        rows = []
        i=1
        model=Db()
        for index, row in data.iterrows():

            rows.append ([f"{row['unique_id']}", f"{row['age']}",\
                      f"{row['workclass']}",f"{row['fnlwgt']}",\
                      f"{row['education']}",f"{row['educationnum']}", \
                      f"{row['marital']}",f"{row['occupation']}", \
                      f"{row['relationship']}",f"{row['race']}", \
                      f"{row['sex']}",f"{row['capitalgain']}", \
                      f"{row['capitalloss']}", f"{row['hoursperweek']}", 
                      f"{row['nativecountry']}"])
                          
            total_records=total_records+1

            if i < records + 1 :   
                i=i+1
            else :
                model.upsert_linkage(rows)
                rows = []
                i=1
                print (f"Ingested {total_records} records")

        if len(rows) > 0 :
            model.upsert_linkage(rows)

        print (f"Ingested {total_records} records")

In [81]:
df = pd.read_csv("adult.data")
df["unique_id"] = df.index

In [82]:
df.columns

Index(['39', ' State-gov', ' 77516', ' Bachelors', ' 13', ' Never-married',
       ' Adm-clerical', ' Not-in-family', ' White', ' Male', ' 2174', ' 0',
       ' 40', ' United-States', ' <=50K', 'unique_id'],
      dtype='object')

In [83]:
df.columns = ["age", "workclass", "fnlwgt", "education", "educationnum", "marital", "occupation", "relationship", "race", "sex", "capitalgain", "capitalloss", "hoursperweek", "nativecountry", "income", "unique_id"]

In [84]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'educationnum', 'marital',
       'occupation', 'relationship', 'race', 'sex', 'capitalgain',
       'capitalloss', 'hoursperweek', 'nativecountry', 'income', 'unique_id'],
      dtype='object')

In [85]:
df.head()

,age,workclass,fnlwgt,education,educationnum,marital,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,nativecountry,income,unique_id
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,1
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,2
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,3
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K,4


In [86]:
#import logging
#logging.basicConfig( level=logging.DEBUG)

import os
import phoenixdb
model=Db()

DEBUG:phoenixdb.avatica.client:Sending request
connection_id: "8fcc8cb3-ec46-462d-939d-59e6947f4010"

DEBUG:phoenixdb.avatica.client:Creating a new Session
DEBUG:phoenixdb.avatica.client:POST https://cod-suflg1bo4gnt-gateway0.demo-aws.ylcu-atmi.cloudera.site/cod-suflg1bo4gnt/cdp-proxy-api/avatica b'\n?org.apache.calcite.avatica.proto.Requests$OpenConnectionRequest\x12&\n$8fcc8cb3-ec46-462d-939d-59e6947f4010' {'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'content-type': 'application/x-google-protobuf'}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cod-suflg1bo4gnt-gateway0.demo-aws.ylcu-atmi.cloudera.site:443
DEBUG:urllib3.connectionpool:https://cod-suflg1bo4gnt-gateway0.demo-aws.ylcu-atmi.cloudera.site:443 "POST /cod-suflg1bo4gnt/cdp-proxy-api/avatica HTTP/1.1" 200 135
DEBUG:phoenixdb.avatica.client:Received response
name: "org.apache.calcite.avatica.proto.Responses$OpenConnectionResponse"
wra

In [87]:
import logging
logging.basicConfig(level=logging.DEBUG)

#model.drop_stocks_table()
model.create_linkage_table()

DEBUG:phoenixdb.avatica.client:Sending request
connection_id: "8fcc8cb3-ec46-462d-939d-59e6947f4010"

DEBUG:phoenixdb.avatica.client:POST https://cod-suflg1bo4gnt-gateway0.demo-aws.ylcu-atmi.cloudera.site/cod-suflg1bo4gnt/cdp-proxy-api/avatica b'\n@org.apache.calcite.avatica.proto.Requests$CreateStatementRequest\x12&\n$8fcc8cb3-ec46-462d-939d-59e6947f4010' {'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'content-type': 'application/x-google-protobuf'}
DEBUG:urllib3.connectionpool:https://cod-suflg1bo4gnt-gateway0.demo-aws.ylcu-atmi.cloudera.site:443 "POST /cod-suflg1bo4gnt/cdp-proxy-api/avatica HTTP/1.1" 200 176
DEBUG:phoenixdb.avatica.client:Received response
name: "org.apache.calcite.avatica.proto.Responses$CreateStatementResponse"
wrapped_message: "\n$8fcc8cb3-ec46-462d-939d-59e6947f4010\020\001\032@\n>cod-suflg1bo4gnt-worker0.demo-aws.ylcu-atmi.cloudera.site:8765"

DEBUG:phoenixdb.avatica.client:Sending reque

In [88]:
import logging
logging.basicConfig(level=logging.DEBUG)

upsert_data(df)

DEBUG:phoenixdb.avatica.client:Sending request
connection_id: "54a76627-e364-41e6-b4b3-9805e8de1e3a"

DEBUG:phoenixdb.avatica.client:Creating a new Session
DEBUG:phoenixdb.avatica.client:POST https://cod-suflg1bo4gnt-gateway0.demo-aws.ylcu-atmi.cloudera.site/cod-suflg1bo4gnt/cdp-proxy-api/avatica b'\n?org.apache.calcite.avatica.proto.Requests$OpenConnectionRequest\x12&\n$54a76627-e364-41e6-b4b3-9805e8de1e3a' {'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'content-type': 'application/x-google-protobuf'}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cod-suflg1bo4gnt-gateway0.demo-aws.ylcu-atmi.cloudera.site:443
DEBUG:urllib3.connectionpool:https://cod-suflg1bo4gnt-gateway0.demo-aws.ylcu-atmi.cloudera.site:443 "POST /cod-suflg1bo4gnt/cdp-proxy-api/avatica HTTP/1.1" 200 135
DEBUG:phoenixdb.avatica.client:Received response
name: "org.apache.calcite.avatica.proto.Responses$OpenConnectionResponse"
wra

ProgrammingError: ('Table undefined. tableName=COD_DEMO_TABLE', 1012, '42M03', None)

In [10]:
model.get_data()

NameError: name 'model' is not defined